<a href="https://colab.research.google.com/github/KRiver28/TIL/blob/master/4_10_sentece_piece.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install sentencepiece

# Google의 Sentencepiece를 이용해서 챗-봇용 학습 문장을 subword token으로
# 분해한다.
#
# 코드 구현 : blog.naver.com/chunjein, 2021.03.31
# ----------------------------------------------------------------------
import pandas as pd
import sentencepiece as spm  # pip install sentencepiece
import re
import pickle

In [18]:
# Commented out IPython magic to ensure Python compatibility.
# 챗-봇 데이터 파일을 읽어온다.
corpus = pd.read_csv('/content/ChatBotData.csv', header=0, encoding='utf-8')

# 질문과 답변을 합쳐서 subword vocabulary를 만든다.
corpusQA = list(corpus['Q']) + list(corpus['A'])

# 특수 문자를 제거한다.
corpusQA = [re.sub("([~.,!?\"':;)(])", "", s) for s in corpusQA]
corpusQA[:10]

corpus.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [21]:
# Sentencepice용 사전을 만들기 위해 corpusQA를 저장해 둔다.
data_file = '/content/ChatBotData.csv'
with open(data_file, 'w', encoding='utf-8') as f:
    for sent in corpusQA:
        f.write(sent + '\n')

# Google의 Sentencepiece를 이용해서 vocabulary를 생성한다.
# -----------------------------------------------------
templates= "--input={0:} \
            --pad_id=0 --pad_piece=<PAD>\
            --unk_id=1 --unk_piece=<UNK>\
            --bos_id=2 --bos_piece=<START>\
            --eos_id=3 --eos_piece=<END>\
            --model_prefix={1:} \
            --vocab_size={2:}"

VOCAB_SIZE = 9600
model_prefix = "/content/chatbot_model"
params = templates.format(data_file, model_prefix, VOCAB_SIZE)


In [22]:
spm.SentencePieceTrainer.Train(params)
sp = spm.SentencePieceProcessor()
sp.Load(model_prefix + '.model')

with open(model_prefix + '.vocab', encoding='utf-8') as f:
    vocab = [doc.strip().split('\t') for doc in f]

word2idx = {k:v for v, [k, _] in enumerate(vocab)}
idx2word = {v:k for k, v in word2idx.items()}

word2idx

{'<PAD>': 0,
 '<UNK>': 1,
 '<START>': 2,
 '<END>': 3,
 '▁': 4,
 '을': 5,
 '가': 6,
 '도': 7,
 '요': 8,
 '이': 9,
 '는': 10,
 '게': 11,
 '▁거예요': 12,
 '은': 13,
 '세요': 14,
 '고': 15,
 '지': 16,
 '를': 17,
 '▁수': 18,
 '의': 19,
 '▁너무': 20,
 '죠': 21,
 '보세요': 22,
 '▁거': 23,
 '▁좋아하는': 24,
 '▁더': 25,
 '▁사람': 26,
 '▁것': 27,
 '▁잘': 28,
 '▁안': 29,
 '서': 30,
 '▁나': 31,
 '한': 32,
 '에': 33,
 '면': 34,
 '▁사랑': 35,
 '네요': 36,
 '▁많이': 37,
 '▁좋은': 38,
 '히': 39,
 '▁그': 40,
 '▁좀': 41,
 '▁싶어': 42,
 '만': 43,
 '▁같아요': 44,
 '▁이별': 45,
 '랑': 46,
 '▁있어요': 47,
 '▁있을': 48,
 '▁사람이': 49,
 '으로': 50,
 '▁게': 51,
 '▁마음': 52,
 '로': 53,
 '네': 54,
 '▁때': 55,
 '▁썸': 56,
 '▁같아': 57,
 '▁내': 58,
 '▁생각': 59,
 '어': 60,
 '다': 61,
 '해보세요': 62,
 '▁다': 63,
 '나': 64,
 '하고': 65,
 '▁마세요': 66,
 '길': 67,
 '▁건': 68,
 '하세요': 69,
 '▁말': 70,
 '▁이제': 71,
 '▁오늘': 72,
 '할': 73,
 '나봐요': 74,
 '▁있': 75,
 '▁좋': 76,
 '▁마음이': 77,
 '▁내가': 78,
 '▁있는': 79,
 '▁왜': 80,
 '에서': 81,
 '야': 82,
 '▁해보세요': 83,
 '▁다른': 84,
 '▁어떻게': 85,
 '겠네요': 86,
 '겠어요': 87,
 '▁남자친구': 88,


In [23]:
# string으로 조회
sentence = corpusQA[610]
enc = sp.encode_as_pieces(sentence)
dec = sp.decode_pieces(enc)

print('\n    문장:', sentence)
print('Subwords:', enc)
print('    복원:', dec)


    문장: 나랑 놀아줘
Subwords: ['▁나랑', '▁놀아줘']
    복원: 나랑 놀아줘


In [24]:
# 수동 decode
print(''.join([x.replace('▁', ' ') for x in enc])[1:])

나랑 놀아줘


In [25]:
# word index로 조회
idx = sp.encode_as_ids(sentence)
dec = sp.decode_ids(idx)

print('\n    문장:', sentence)
print('Subwords:', idx)
print('    복원:', dec)


    문장: 나랑 놀아줘
Subwords: [1070, 3123]
    복원: 나랑 놀아줘


In [26]:
# word index로 변환한다.
corpusQA_idx = [sp.encode_as_ids(qa) for qa in corpusQA]
corpusQA_idx[0]

[4312, 358, 2447]

In [27]:
corpusQA[0]

'12시 땡'

In [28]:
idx2word[4312]

'▁12'